In [34]:
!pip install shap


In [35]:
!pip install pycaret


In [36]:
!pip install ipywidgets

In [37]:
#%%
import pandas as pd
from pandas.plotting import scatter_matrix
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import shap
import pycaret
#%%

In [38]:
# 데이터 불러오기
file_id = "1z7RNDuXOGAhbEv94SyzV2kR7dTF6lXr5"

data_url = f"https://drive.google.com/uc?export=download&id={file_id}"

df_raw = pd.read_csv(data_url, sep=None, engine="python", on_bad_lines="skip")

In [39]:
# 긍정, 부정 변경
df_raw.info()

df = df_raw
#%%
cols = [
    "wstat4", "wstat6",
    "wwa1", "wwa2", "wwa3", "wwa4", "wwa5",
    "sleep1", "sleep2", "sleep3",
    "wsituation12", "wsituation14"
]

# 변환 딕셔너리 (5점 척도 역코딩)
reverse_map = {1: 5, 2: 4, 3: 3, 4: 2, 5: 1}

# 여러 열에 적용
for col in cols:
    df_raw[col] = df_raw[col].map(reverse_map).fillna(df_raw[col])


#%%
print(df_raw.columns.tolist())
#%%
print(df_raw[cols].head())
#%%
df["wstat4"].head(10) 
# %%

cols_1 = [
    "imte1", "imte2", "imte3", "imte4", "imte5",
]

# 변환 딕셔너리 (4점 척도 역코딩)
reverse_map = {1: 4, 2: 3, 3: 2, 4: 1}

# 여러 열에 적용
for col in cols_1:
    df_raw[col] = df_raw[col].map(reverse_map).fillna(df_raw[col])


#%%
print(df_raw.columns.tolist())
#%%
print(df_raw[cols_1].head())
#%%
df["imte1"].head(10) 
#%%

cols_2 = [
   "heal_lim1"
]

# 변환 딕셔너리 (3점 척도 역코딩)
reverse_map = {1: 3, 2: 2, 3: 1}

# 여러 열에 적용
for col in cols_2:
    df_raw[col] = df_raw[col].map(reverse_map).fillna(df_raw[col])


#%%
print(df_raw.columns.tolist())
#%%
print(df_raw[cols_2].head())
#%%
df["heal_lim1"].head(10) 
#%%

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10173 entries, 0 to 10172
Columns: 232 entries, id to hm_01_rel_t
dtypes: float64(1), int64(229), object(2)
memory usage: 18.0+ MB
['id', 'burnout60', 'area', 'gender', 'age', 'wduration_y', 'earning1', 'earning2_r', 'wtime_r', 'woutside4_a', 'wtime_length5', 'age_group', 'emp_period_group', 'wtime_group', 'compsize_group', 'woutside_group', 'compsize_group_a', 'wday_week_group_a', 'woutside4_group_a', 'occ_t2_group_a', 'earning2_group_b', 'occ_group_a', 'emp_stat', 'emp_con_term', 'emp_keep', 'emp_own_mgmt', 'emp_fptime', 'emp_wage', 'emp_stat_sp', 'income_pos_a', 'income_pos_b', 'income_pos_c', 'income_pos_d', 'income_pos_e', 'income_pos_f', 'income_pos_g', 'income_pos_h', 'income_pos_i', 'income_pos_j', 'emp_place', 'emp_suggest_a', 'emp_suggest_b', 'emp_boss_gender', 'emp_manaqual_a', 'emp_manaqual_b', 'emp_manaqual_c', 'emp_manaqual_d', 'emp_manaqual_e', 'emp_comp_ass_a', 'emp_comp_ass_b', 'emp_comp_ass_c', 'emp_comp_ass_d', 'emp_c

0    1
1    1
2    1
3    1
4    1
5    1
6    1
7    1
8    1
9    1
Name: heal_lim1, dtype: int64

In [40]:
# 불필요한 피쳐 삭제하기 (14개)
df = df.drop(columns=["ptime_r", "stratification", "district", "household", "target", "mode", "hh_num", "eli_num", "hm_01_gender", "hm_01_year", "hm_01_estat", "hm_01_rel_t","wtime_week", "earning1_r"])
df

,id,burnout60,area,gender,age,wduration_y,earning1,earning2_r,wtime_r,woutside4_a,...,edu,marital,children,panel_survey,ind,ind2,occ,occ2,comp_sizea_r,comp_sizeb_r
0,1000042_1,lv1,1,1,49,10,500,6,40,1,...,5,1,1,2,7,46,3,313,4,4
1,1000051_1,lv2,1,1,31,2,300,4,48,1,...,6,2,2,2,7,47,3,312,11,11
2,1000056_1,lv1,1,1,32,2,300,4,50,1,...,5,2,2,2,9,56,4,441,2,2
3,1000069_1,lv1,1,1,38,2,390,4,40,1,...,6,1,1,2,6,42,3,312,10,10
4,1000069_2,lv1,1,2,37,5,330,4,40,1,...,6,1,2,2,13,71,2,273,7,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10168,2009807_1,lv1,13,1,31,1,250,3,48,1,...,6,2,2,2,3,28,8,831,7,7
10169,2010133_1,lv2,7,2,46,3,230,3,40,1,...,5,2,2,2,8,52,3,313,4,4
10170,2010134_2,lv1,7,2,34,5,320,4,40,1,...,6,2,2,2,11,64,3,320,4,4
10171,2010137_1,lv1,7,1,57,5,450,5,40,1,...,6,1,1,2,7,46,5,510,4,4


In [41]:
# 전처리: 8, 9 NULL (5개)
cols = ["hazard_erg_a", "hazard_erg_b", "hazard_erg_c", "hazard_erg_d", "hazard_erg_e"]

df[cols] = df[cols].replace([8, 9], pd.NA)

# NA가 하나라도 있는 행 삭제
df = df.dropna()
print("삭제 전 행 개수:", len(df_raw))
print("삭제 후 행 개수:", len(df))

삭제 전 행 개수: 10173
삭제 후 행 개수: 10157


In [42]:
# 잘못 변경된 변수면 복원
df.rename(columns={
    "wsituation": "wsituation8",
    "wsituation.1": "wsituation9",
    "disc": "disc9",
    "disc.1": "disc8"
}, inplace=True)

In [43]:
from sklearn.preprocessing import MinMaxScaler

# 정규화할 열 리스트 
ordinal_cols = ["emp_manaqual_a",
	"emp_manaqual_b",
	"emp_manaqual_c",
	"emp_manaqual_d",
	"emp_manaqual_e",
	"emp_comp_ass_a",
	"emp_comp_ass_b",
	"emp_comp_ass_c",
	"emp_comp_ass_d",
	"emp_comp_ass_e",
	"emp_comp_ass_f",
	"emp_winflu_a",
	"emp_winflu_b",
	"emp_winflu_c",
	"imte1",
	"imte2",
	"imte3",
	"imte4",
	"imte5",
	"safeinform",
	"wsituation1",
	"wsituation2",
	"wsituation3",
	"wsituation4",
	"wsituation5",
	"wsituation11",
	"wsituation6",
	"wsituation7",
	"wsituation8",
	"wsituation9",
	"wsituation10",
	"wsituation12",
	"wsituation13",
	"wsituation14",
	"wstat1",
	"wstat2",
	"wstat3",
	"wstat4",
	"wstat5",
	"wstat6",
	"wstat7",
	"wtime_arr_a",
	"wtime_ftwork",
	"wtime_ftcomtool",
	"wtime_private",
	"hazard_phy_b",
	"hazard_erg_a",
	"hazard_erg_b",
	"hazard_erg_c",
	"hazard_erg_d",
	"hazard_erg_e",
	"winten2_a",
	"winten2_b",
	"winterrupt1",
	"weng1",
	"weng2",
	"weng3",
	"weng4",
	"weng5",
	"satisfaction",
	"wplace4",
	"wplace3",
	"wplace2",
	"wplace1",
	"wpalce_ch",
	"wbalance",
	"wwa1",
	"wwa2",
	"wwa3",
	"wwa4",
	"wwa5",
	"heal_cond",
	"heal_lim1",
	"sleep1",
	"sleep2",
	"sleep3",
	"who1",
	"who2",
	"who3",
	"who4",
	"who5",
	"woutside3",
	"woutside4",
	"woutside5",
	"income_bal"
	] 

# Min-Max 정규화 객체 생성
scaler = MinMaxScaler()

# 서열척도 열만 정규화
df[ordinal_cols] = scaler.fit_transform(df[ordinal_cols])

In [44]:
# 범주화 

#%%
print(df_raw.columns.tolist())
#%%
print(df_raw[cols].head())
#%%
df["wstat4"].head(10) 
#%%
df_raw["heal_abs1_group"] = np.where(df_raw["heal_abs1"] == 0, 0, 1)
#%%
print(df_raw["heal_abs1_group"].head())
#%%
df_raw["heal_abs1_group"].value_counts()
#%%

# 구간 (0~60, 61~120, 181 이상)
bins = [0, 60, 120, float("inf")]
labels = [1, 2, 3]

df_raw["ctime_group"] = pd.cut(df_raw["ctime"], bins=bins, labels=labels, right=True)
#%%
print(df_raw["ctime_group"].head())
#%%
df_raw["ctime_group"].value_counts()
#%%

['id', 'burnout60', 'area', 'gender', 'age', 'wduration_y', 'earning1', 'earning2_r', 'wtime_r', 'woutside4_a', 'wtime_length5', 'age_group', 'emp_period_group', 'wtime_group', 'compsize_group', 'woutside_group', 'compsize_group_a', 'wday_week_group_a', 'woutside4_group_a', 'occ_t2_group_a', 'earning2_group_b', 'occ_group_a', 'emp_stat', 'emp_con_term', 'emp_keep', 'emp_own_mgmt', 'emp_fptime', 'emp_wage', 'emp_stat_sp', 'income_pos_a', 'income_pos_b', 'income_pos_c', 'income_pos_d', 'income_pos_e', 'income_pos_f', 'income_pos_g', 'income_pos_h', 'income_pos_i', 'income_pos_j', 'emp_place', 'emp_suggest_a', 'emp_suggest_b', 'emp_boss_gender', 'emp_manaqual_a', 'emp_manaqual_b', 'emp_manaqual_c', 'emp_manaqual_d', 'emp_manaqual_e', 'emp_comp_ass_a', 'emp_comp_ass_b', 'emp_comp_ass_c', 'emp_comp_ass_d', 'emp_comp_ass_e', 'emp_comp_ass_f', 'emp_rep', 'emp_winflu_a', 'emp_winflu_b', 'emp_winflu_c', 'emp_tra_b', 'emp_tra_c', 'emp_tra_d', 'wsector', 'comp_size_a', 'comp_size_b', 'comp_female

ctime_group
1    8556
2     851
3     241
Name: count, dtype: int64

In [45]:
# 데이터 저장
df.to_csv("2PD_v1_0825.csv", index=False, encoding="utf-8-sig")